In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import datasets, transforms
import helper
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report,accuracy_score
from google.colab import files
from torchsummary import summary
import time
import copy
# import onnx
# from onnx2pytorch import ConvertModel
from tqdm import trange
from tqdm import tqdm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

criterion = nn.CrossEntropyLoss()
epoch = 100


In [4]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
# build dataloader
mean = torch.tensor([135.31470632, 124.53849418, 103.39646082])
std = torch.tensor([5.18153318, 4.14170719, 5.17011963])

transform = transforms.Compose([transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.RandomRotation(degrees=360),
                                transforms.RandomHorizontalFlip(p=0.5),
                                transforms.ColorJitter(brightness=.5, hue=.3),
                                transforms.RandomInvert(),
                                transforms.Normalize(mean, std)])

# transform = transforms.Compose([transforms.Resize(255),
#                                 transforms.CenterCrop(224),
#                                 transforms.ToTensor(),
#                                 transforms.Normalize(mean, std)])

balanced_path_data = 'drive/MyDrive/balanced_csic_data_64/training'


def train_val_dataset(dataset, val_split=0.1):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets

# load the balanced dataset
balanced_dataset = datasets.ImageFolder(balanced_path_data, transform=transform)
balanced_datasets = train_val_dataset(balanced_dataset)


class_dict = balanced_dataset.class_to_idx

In [6]:
batch_size = 32
balanced_dataloaders_dict = {x: torch.utils.data.DataLoader(balanced_datasets[x], batch_size=batch_size, shuffle=True, num_workers=2) for x in ['train', 'val']}

In [11]:
def train_model(model, dataloaders, criterion, optimizer,lr_scheduler,  num_epochs=20, is_inception=False):
    since = time.time()

    val_acc_history = []
    train_acc_history = []
    lrs = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            with tqdm(dataloaders[phase],unit = 'batch') as tepoch:
              # Iterate over data.
              for inputs, labels in tepoch:
                  inputs = inputs.to(device)
                  labels = labels.to(device)

                  # zero the parameter gradients
                  optimizer.zero_grad()

                  # forward
                  # track history if only in train
                  with torch.set_grad_enabled(phase == 'train'):
                      # Get model outputs and calculate loss
                      # Special case for inception because in training it has an auxiliary output. In train
                      #   mode we calculate the loss by summing the final output and the auxiliary output
                      #   but in testing we only consider the final output.
                      if is_inception and phase == 'train':
                          # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                          outputs, aux_outputs = model(inputs)
                          loss1 = criterion(outputs, labels)
                          loss2 = criterion(aux_outputs, labels)
                          loss = loss1 + 0.4*loss2
                      else:
                          outputs = model(inputs)
                          loss = criterion(outputs, labels)

                      _, preds = torch.max(outputs, 1)
                      
                      
                      # backward + optimize only if in training phase
                      if phase == 'train':
                          loss.backward()
                          optimizer.step()
                          

                  # statistics
                  running_loss += loss.item() * inputs.size(0)
                  #getCategoricalAccuracy(preds, labels.data, class_dict)
                  running_corrects += torch.sum(preds == labels.data)

              epoch_loss = running_loss / len(dataloaders[phase].dataset)
              epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

              print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
              if phase == 'train':
                lrs.append(optimizer.param_groups[0]["lr"])
                scheduler.step()
                print("Current Learning Rate = ",optimizer.param_groups[0]["lr"])


              # deep copy the model
              if phase == 'val' and epoch_acc > best_acc:
                  best_acc = epoch_acc
                  best_model_wts = copy.deepcopy(model.state_dict())
              if phase == 'val':
                  val_acc_history.append(epoch_acc)
              if phase == 'train':
                  train_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, train_acc_history

In [9]:
# ResNet 50
feature_extract = True
model_ft_balanced = models.resnet50(pretrained=False)
model_ft_balanced.fc = nn.Linear(2048, len(balanced_dataset.class_to_idx))
input_size = 224
model_ft_balanced = model_ft_balanced.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft_balanced.parameters()
# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(params_to_update, lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.5)
# Training a pretrained Resnet18 on a balanced dataset
model_ft_balanced, balanced_val_hist, balanced_tr_hist  = train_model(model_ft_balanced, balanced_dataloaders_dict, criterion, optimizer_ft,scheduler, num_epochs= epoch)


Epoch 0/99
----------


100%|██████████| 94/94 [17:16<00:00, 11.02s/batch]


train Loss: 1.0325 Acc: 0.5000
Current Learning Rate =  0.0001


100%|██████████| 11/11 [01:53<00:00, 10.32s/batch]


val Loss: 1.3133 Acc: 0.4880

Epoch 1/99
----------


...

100%|██████████| 94/94 [00:44<00:00,  2.13batch/s]


train Loss: 0.5495 Acc: 0.7898
Current Learning Rate =  1.0000000000000008e-14


100%|██████████| 11/11 [00:04<00:00,  2.37batch/s]

val Loss: 0.5846 Acc: 0.7651

Training complete in 99m 36s
Best val Acc: 0.816265


ValueError: ignored

In [25]:
#ResNet 152
model_ft_balanced3 = models.resnet152(pretrained=False)
set_parameter_requires_grad(model_ft_balanced3, True)
model_ft_balanced3.fc = nn.Linear(2048, len(balanced_dataset.class_to_idx))
input_size = 224
model_ft_balanced3 = model_ft_balanced3.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft_balanced3.parameters()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(params_to_update, lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

# Training a pretrained Resnet50 on a balanced dataset
model_ft_balanced3, balanced_val_hist3, balanced_tr_hist3  = train_model(model_ft_balanced3, balanced_dataloaders_dict, criterion, optimizer_ft,scheduler,num_epochs= epoch)


Epoch 0/99
----------


100%|██████████| 94/94 [00:48<00:00,  1.95batch/s]


train Loss: 3.9342 Acc: 0.4240
Current Learning Rate =  0.01


100%|██████████| 11/11 [00:05<00:00,  2.05batch/s]


val Loss: 3.0651 Acc: 0.3645

Epoch 1/99
----------


...

val Loss: 1.0411 Acc: 0.4608

Epoch 99/99
----------


100%|██████████| 94/94 [00:41<00:00,  2.29batch/s]


train Loss: 0.9991 Acc: 0.4953
Current Learning Rate =  1.0000000000000006e-12


100%|██████████| 11/11 [00:06<00:00,  1.77batch/s]


val Loss: 1.0367 Acc: 0.4819

Training complete in 82m 52s
Best val Acc: 0.536145


In [17]:
test_path_data = 'drive/MyDrive/balanced_csic_data_64/testing'
testing_dataset = torchvision.datasets.ImageFolder(test_path_data, transform=transform)
batch_size = 10
testing_dataloader =torch.utils.data.DataLoader(testing_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
print(testing_dataset.class_to_idx)

{'Cyperus': 0, 'Negative': 1, 'Portulaca': 2, 'Solanum': 3}


In [18]:
def getTestingMetrics(dataloader, model):
  # send model to GPU
  if torch.cuda.is_available():
    model.cuda()

  # with tqdm(testing_dataloader) as tepoch:
  lbs = []
  preds = []
  #   # Iterate over data.
  for inputs, labels in dataloader:
    test_inputs = inputs.to(device)
    #test_labels = labels.to(device)
    outputs = model(test_inputs)
    _, pred = torch.max(outputs, 1)
    lbs+=(list(labels.numpy()))
    preds+= list(pred.cpu().data.numpy())
  print('Overall testing accuracy:',accuracy_score(lbs, preds))
  print(classification_report(lbs, preds, target_names=class_dict.keys()))


In [19]:
getTestingMetrics(testing_dataloader, model_ft_balanced)

a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
Overall testing accuracy: 0.7376470588235294
              precision    recall  f1-score   support

     Cyperus       0.69      0.75      0.72       302
    Negative       0.78      0.90      0.84       425
   Portulaca       0.00      0.00      0.00        85
     Solanum       0.56      0.47      0.51        38

    accuracy                           0.74       850
   macro avg       0.51      0.53      0.52       850
weighted avg       0.66      0.74      0.70       850



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
getTestingMetrics(testing_dataloader, model_ft_balanced3)

a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
Overall testing accuracy: 0.4964705882352941
              precision    recall  f1-score   support

     Cyperus       0.41      0.28      0.34       302
    Negative       0.53      0.79      0.63       425
   Portulaca       0.00      0.00      0.00        85
     Solanum       0.00      0.00      0.00        38

    accuracy                           0.50       850
   macro avg       0.23      0.27      0.24       850
weighted avg       0.41      0.50      0.43       850



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
